# 实例简介
## 欧拉放大介绍
EVM(Eulerian Video Magnification,欧拉视频放大或欧拉影像放大)是一种将视频中的微小变化进行放大的算法，该算法可以将视频中的微小变化转变为肉眼可以观察的变化。这个算法可以应用在从视频中提取心率信息，也可以放大视频中细小的运动变化。
## Spatial Decomposition
将视频中的每一帧进行下采样，形成图像金字塔。这里图像金字塔有两种组件方式，一种是高斯金字塔，还有一种是拉普拉斯金字塔。高斯金字塔常常用于放大色彩，而拉普拉斯金字塔用于放大运动。
## Temporal Filter
视频可以看做连续的图片，从图片中的单个像素点的角度看，视频每个像素点的变化可以看成时域信号。而物体运动的信息就隐藏在单个像素点的变化之中。论文采用的方法是进行带通滤波（bandpass filter），放大颜色的时候采用理想带通滤波器，放大运动的时候采用IIR滤波器，论文中主要使用了巴特沃斯滤波器（Butterworth filter）。
## Reconstraction
最后一步流程就是将分解成图像金字塔的图片再复原回去。这篇论文中，高斯金字塔的复原方法是将高斯金字塔中最低一级（图片最小的那张）进行上采样，然后叠加到原图中。在工程实现中，最后图片的像素值会超过255，发生溢出，此时需要做的是把像素值归一化到0-255之间。应该注意到，这里与原论文的描述不完全一样。对于拉普拉斯金字塔，先将金字塔最低一级进行上采样，然后与上一级进行叠加，以此类推，最后与原图叠加，这里的方法与论文本身基本一致。

# Coding

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# 读取视频并分离每一帧数据，由于内存问题这里可以限制视频的帧数
def separateVideoFrameToDisk(videoname, path, limitedCount = 300):
    frames = [ ]
    os.makedirs(path, exist_ok = True)
    cap = cv2.VideoCapture(videoname)
    if not cap.isOpened():
        return None
    fps = cap.get(cv2.CAP_PROP_FPS)
    ret, image = cap.read()
    count = 0
    ret = True
    while ret and count < limitedCount:
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #cv2.imwrite(os.path.join(path, "frame%d.jpg" % count), image)
        frames.append(image)
        ret, image = cap.read()
        count += 1
    frames = np.array(frames)
    cap.release()
    return fps, frames

In [ ]:
# 分析视频为图片
#fps, frames = separateVideoFrameToDisk('data/face.mp4', 'frames/face')
fps, frames = separateVideoFrameToDisk('data/baby2.mp4', 'frames/baby2')
#fps, frames = separateVideoFrameToDisk('data/baby.mp4', 'frames/baby', 100)

In [ ]:
frameNum, H, W, chNum = frames.shape
print(frameNum, W, H, chNum)
print(frames.shape)

In [ ]:
# 画出某个像素时域信号变化情况
def draw1DTemporalSignal(frames, ypos, xpos, fps = 30):
    frameNum, H, W, chNum = frames.shape
    tlist = [t * 1000 / fps for t in range(frameNum)]
    #print(tlist)
    #print(len(frames[:, ypos, xpos, 2]))
    plt.figure(figsize=(20,5))
    channelName=['R', 'G', 'B']
    for c in range(chNum):
        plt.subplot(1, 3, c+1)
        plt.plot(tlist, frames[:, ypos, xpos, c], '-g')
        plt.title('%s channel pixel value change for point (%d, %d)' % (channelName[c], ypos, xpos))
        plt.xlabel('Time(ms)')
        plt.ylabel('Pixel Value')
    plt.show()

In [ ]:
draw1DTemporalSignal(frames, 100, 100, fps)

* map所有的视频帧像素值到[0..1]，归一化处理

In [ ]:
# 归一化操作
def normalization(x,Max,Min):
    y = np.ndarray(shape=x.shape, dtype=np.float64)
    y = (x - Min) / (Max - Min);
    return y

frames_normal = np.ndarray(shape=frames.shape, dtype=np.float64)
for frame,n in zip(frames, range(frameNum)):
    img = normalization(frame, 255, 0)
    frames_normal[n] = img
del frames

In [ ]:
#convert RGB to YIQ
def rgb2ntsc(R,G,B):
    Y = 0.299*R + 0.587*G + 0.114*B
    I = 0.596*R - 0.274*G - 0.322*B
    Q = 0.211*R - 0.523*G + 0.312*B
    return Y,I,Q

#convert YIQ to RGB
def ntsc2rgb(Y,I,Q):
    R = 1.000*Y + 0.956*I + 0.621*Q
    G = 1.000*Y + -0.272*I + -0.647*Q
    B = 1.000*Y + -1.106*I + 1.703*Q
    return R,G,B

def convert_RGB_frame_to_YIQ(frame):
    dtype = frame.dtype
    result = np.ndarray(shape=frame.shape, dtype=dtype)
    np.copyto(result, frame)
    #R,G,B = extract_tuples(frame)
    R = frame[:,:,0]
    G = frame[:,:,1]
    B = frame[:,:,2]
    Y,I,Q = rgb2ntsc(R,G,B)
    result[:,:,0] = Y
    result[:,:,1] = I
    result[:,:,2] = Q
    return result

def convert_YIQ_frame_to_RGB(frame):
    dtype = frame.dtype
    result = np.ndarray(shape=frame.shape, dtype=dtype)
    np.copyto(result, frame)
    Y = frame[:,:,0]
    I = frame[:,:,1]
    Q = frame[:,:,2]
    R,G,B = ntsc2rgb(Y,I,Q)
    result[:,:,0] = R 
    result[:,:,1] = G 
    result[:,:,2] = B 
    return result

In [ ]:
# 验证YIQ转换是否能逆变换回RGB图
def checkYIQTransform(YIQimg):
    RGBImg = convert_YIQ_frame_to_RGB(YIQimg)
    print(RGBimg)
    RGBImg *= 255
    RGBImg=RGBImg.astype(np.uint8)
    print(RGBimg)
    RGBImg[RGBimg>255] = 255
    RGBImg[RGBimg<0] = 0
    cv2.imwrite(os.path.join('frames', "rgb.jpg" ), RGBimg)

* 转换所有的视频frame到YIQ色彩空间

In [ ]:
for frame,i in zip(frames_normal, range(frameNum)):
    img = convert_RGB_frame_to_YIQ(frame)
    frames_normal[i] = img

* 拉普拉斯金字塔构建

In [ ]:
def build_lappyr(frameYIQ, leveln = 4):
    lappyrs = []
    img = frameYIQ
    for i in range(leveln-1):
        next_img = cv2.pyrDown(img)
        (h, w) = img.shape[:2]
        img1 = cv2.pyrUp(next_img, dstsize=(w, h))
        #print((img-img1).shape)
        lappyrs.append(img-img1)
        img = next_img
    #print(img.shape)
    lappyrs.append(img)
    return np.array(lappyrs)

In [ ]:
def laplacian_video(frames,levels=4):
    print("  laplacian_video E")
    lappyrAll=[]
    for i in range(0, frames.shape[0]):
        frame=frames[i]
        pyr=build_lappyr(frame,levels)
        for n in range(levels):
            if i==0:
                lappyrAll.append(np.zeros((frames.shape[0], pyr[n].shape[0], pyr[n].shape[1],3), dtype="float64"))
            lappyrAll[n][i] = pyr[n]
    print("  laplacian_video X")
    return lappyrAll

lappyrAll = laplacian_video(frames_normal)
del frames_normal

* 带通滤波器实现

In [ ]:
import scipy.fftpack as fftpack
def temporal_bandpass_filter(data, fps, freq_min=0.833, freq_max=1, axis=0, amplification_factor=1):
    """Found from https://github.com/brycedrennan/eulerian-magnification. Will expand later."""
    fft = fftpack.rfft(data, axis=axis)
    frequencies = fftpack.fftfreq(data.shape[0], d=1.0 / fps)
    bound_low = (np.abs(frequencies - freq_min)).argmin()
    bound_high = (np.abs(frequencies - freq_max)).argmin()
    fft[:bound_low] = 0 
    fft[bound_high:-bound_high] = 0 
    fft[-bound_low:] = 0 

    result = np.ndarray(shape=data.shape, dtype='float')
    result[:] = fftpack.ifft(fft, axis=0)
    result *= amplification_factor
    return result

def filter_lap_pyramid(video_pyramid, fps, freq_min=0.833, freq_max=1, axis=0, amplification_factor=1):
    print("filter_lap_pyramid E")
    #print("video_pyramid = ", len(video_pyramid))
    for i in range(0, len(video_pyramid)):
        video = video_pyramid[i]
        filtered_video = temporal_bandpass_filter(video, fps, freq_min, freq_max, axis, amplification_factor)
        video_pyramid[i] += filtered_video
        print(filtered_video.shape)
    print("filter_lap_pyramid X")
    return video_pyramid

In [ ]:
filteredPyrs = filter_lap_pyramid(lappyrAll, fps, 0.4, 3, 0, 10)
del lappyrAll

* 利用过滤并完成放大的拉布拉斯金字塔从新还原视频

In [ ]:
def reconstract_video(filteredPyramid, pyramidLevels = 4):
    print("reconstructePyramid E")
    level = pyramidLevels - 1
    while ( level >= 1):
        for frameNum in range(len(filteredPyramid[level])):
            filteredPyramid[level - 1][frameNum] = cv2.pyrUp(filteredPyramid[level][frameNum]) + filteredPyramid[level - 1][frameNum]
        level -= 1
    reconstructedData = np.array(filteredPyramid[0])

    print("reconstructePyramid X")
    return reconstructedData

In [ ]:
newYIQFrames = reconstract_video(filteredPyrs)
del filteredPyrs
newRGBFrames = np.ndarray(shape=newYIQFrames.shape, dtype=np.float64)
for frame,i in zip(newYIQFrames, range(frameNum)):
    img = convert_YIQ_frame_to_RGB(frame)
    newRGBFrames[i] = img
del newYIQFrames

* 保存还原后的frame为结果视频

In [ ]:
def saveVideo(frames, output,width,height):
    print("saveVideo E")
    fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
    writer = cv2.VideoWriter("result/"+output+".avi", fourcc, 30, (width, height), 1)
    for frame in frames:
        frame *= 255
        frame[frame>255] = 255
        frame[frame<0] = 0
        writer.write(cv2.convertScaleAbs(frame))
    writer.release()
    print("saveVideo X")
    return None

In [ ]:
#saveVideo(newRGBFrames, "face", W, H)
saveVideo(newRGBFrames, "baby2", W, H)
#saveVideo(newRGBFrames, "baby", W, H)